In [1]:
import visualisation as v
import data_handling as dh
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import plotly.graph_objects as go

db = dh.DataBase()


c:\Users\archie.rowberry\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\archie.rowberry\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\archie.rowberry\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [ ]:
folders = [x[1] for x in os.walk("data/")][0]
for folder in folders:
    print(folder)
    file = db.find_filenames("data/{}/".format(folder))[0]
    db.load_df("data/{}/{}".format(folder, file))
    db.store_df(file[:-4])

In [70]:
from time import sleep


tests = db.collection.list_items()

test = list(tests)[0]
for test in tqdm(tests):
    df = dh.Test(db, test)
    df = df.data

    max_R = df.groupby(["Puff Count"]).max()
    min_R = df.groupby(["Puff Count"]).min()

    vis = v.Vis(test)

    fig = go.Figure()

    fig.add_trace(go.Scattergl(x=max_R.index, y=max_R["Resistance End (Ω)"],
                               name="Maximum Resistance of Puff", 
                            #    mode='markers', 
                            #    marker=dict(size=7, 
                            #                color=list(map(vis.SetColor, max_R["Resistance End (Ω)"].to_list()))
                            #               )
                              )
                 )
    fig.add_trace(go.Scattergl(x=min_R.index, y=min_R["Resistance Start (Ω)"], name="Minimum Resistance of Puff"))

    fig.add_trace(go.Scattergl(x=[0, list(max_R.index)[-1]*2],
                               y=[max_R["Resistance End (Ω)"].max(), max_R["Resistance End (Ω)"].max()],
                               name="Maximum Resistance of Test", mode='lines', 
                               line=dict(dash='dash')))
    fig.add_annotation(x=0.9*max(list(max_R.index)), y=max_R["Resistance End (Ω)"].max()+0.01,
                    text="Maximum Test Resistance: {}".format(round(max_R["Resistance End (Ω)"].max(), 3)),
                    showarrow=False,
                    )

    fig.add_trace(go.Scattergl(x=[0, list(max_R.index)[-1]*2], y=[1.29*min_R["Resistance Start (Ω)"].min(), 1.29*min_R["Resistance Start (Ω)"].min()], name="IQOS Resistance Limit", mode='lines', line=dict(dash='dash')))
    fig.add_annotation(x=0.6*max(list(max_R.index)), y=1.29*min_R["Resistance Start (Ω)"].min()+0.01,
                    text="IQOS Resistance Limit: {}".format(round(1.3*min_R["Resistance Start (Ω)"].min(), 3)),
                    showarrow=False,
                    )


    fig.update_xaxes(range=[-5, 1.05*max(list(max_R.index))])

    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

    fig.update_layout(
        xaxis_title="Puff Count",
        yaxis_title="Resistance [Ω]",
        title=test,
    )

    fig.update_layout(legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99))
    fig.update_layout(
        width=1500,
        height=700,
        template="plotly_white",
        margin=dict(l=40, r=10, t=40, b=40)
    )

    fig.write_image("docs/images/{}.png".format(test), scale=3)



    first_pulses = df.loc[df["dR/dt (Ω/s)"] <= 5]
    dR_df = first_pulses.groupby("Puff Count")["dR/dt (Ω/s)"].mean()
    dR_df["SMA 10"] = dR_df["dR/dt (Ω/s)"]


    fig = go.Figure()

    

    fig.update_xaxes(range=[-5, 1.05*max(list(max_R.index))])
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.update_layout(
        xaxis_title="Puff Count",
        yaxis_title="Resistance [Ω]",
        title=test,
    )
    fig.update_layout(legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99))
    fig.update_layout(
        width=1500,
        height=700,
        template="plotly_white",
        margin=dict(l=40, r=10, t=40, b=40)
    )


        


100%|██████████| 12/12 [00:12<00:00,  1.00s/it]


In [77]:
first_pulses = df.loc[df["dR/dt (Ω/s)"] <= 5]
dR_df = first_pulses.groupby("Puff Count")["dR/dt (Ω/s)"].mean()
dR_df = pd.DataFrame({"dR/dt (Ω/s)": dR_df})
dR_df["SMA 10"] = dR_df["dR/dt (Ω/s)"].rolling(window=10, min_periods=0, center=False).mean()
dR_df["SMA 40"] = dR_df["dR/dt (Ω/s)"].rolling(window=40, min_periods=0, center=False).mean()

,dR/dt (Ω/s),SMA 10
Puff Count,,
1.0,3.026542,3.026542
2.0,3.022507,3.024524
3.0,3.000006,3.016352
4.0,2.951274,3.000082
5.0,2.932256,2.986517
...,...,...
934.0,2.379873,2.278935
935.0,2.400437,2.283454
936.0,2.117711,2.258306
